In [ ]:
# imports
import pandas
import geopandas
import shapely

In [ ]:
monasteries = pandas.read_csv('../data/monasteries.csv', converters={'geometry': shapely.wkt.loads}, dtype={'foundingYear': int})
monasteries = geopandas.GeoDataFrame(monasteries, crs=4326)
monasteries.head()

In [ ]:
breweries = pandas.read_csv('../data/breweries.csv', converters={'geometry': shapely.wkt.loads, 'foundingYear': lambda x: int(x) if x else x})
breweries = geopandas.GeoDataFrame(breweries, crs=4326)
breweries.head()

In [ ]:
monasteries.explore(
    color = "cornflowerblue", 
    marker_kwds=dict(radius=5),
    popup =True,    
    width =800,
    height = 500
    )


In [ ]:
map = monasteries.explore(
    color = "cornflowerblue", 
    marker_kwds=dict(radius=5), 
    popup =True,
    width =800,
    height = 500
    )
breweries.explore(
    m = map, 
    color = "tomato", 
    marker_kwds=dict(radius=3), 
    popup =True
    )

In [ ]:
min_dist = geopandas.sjoin_nearest(left_df = breweries, right_df=monasteries, distance_col="distance")
min_dist.head()

In [ ]:
breweries_projected = breweries.to_crs("EPSG:25832")
monasteries_projected = monasteries.to_crs("EPSG:25832")

In [ ]:
min_dist = geopandas.sjoin_nearest(left_df = breweries_projected, right_df=monasteries_projected, distance_col="distance[m]")
min_dist.head()

In [ ]:
min_dist['distance[m]'].plot.hist(bins=100)

In [ ]:
# reorder columns
min_dist = min_dist.reindex(columns=['brewery', 'monastery', 'distance[m]', 'foundingYear_left', 'foundingYear_right', 'geometry'])
# rename founding year columns
min_dist = min_dist.rename(columns={'foundingYear_left': 'founding_brewery', 'foundingYear_right': 'founding_monastery'})
# sort and reindex
min_dist = min_dist.sort_values(by='distance[m]').reset_index().drop(columns='index')
min_dist.head(n=20)

In [ ]:
min_dist.explore(
    column='distance[m]', 
    style_kwds={"style_function": lambda x: {"radius": x["properties"]["distance[m]"]/1000}}, 
    popup =True,    
    width =800,
    height = 500
    )


In [ ]:
min_dist = min_dist.sort_values(by='distance[m]', ascending=False).reset_index().drop(columns='index')
min_dist.explore(
    column='distance[m]', 
    style_kwds={"style_function": lambda x: {"radius": (x["properties"]["distance[m]"]/1000)+1}},  
    popup =True,
    width =800,
    height = 500,
    cmap = "magma_r",
    tiles="Stamen Toner"
    )

# colormaps in Python Matplotlib: https://matplotlib.org/stable/tutorials/colors/colormaps.html